<a href="https://colab.research.google.com/github/firestar2202/AOA-Graph-Representation/blob/master/gpu_population_dynamics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PS1: GPU Population Dynamics Simulation (10 Points)

## Problem Statement
In this assignment you are going to build a simulation for population dynamics using C++ in a multithreaded environment. We have provided some starter code for you, and your job is to fill in the missing code as specified by the `#TODO#` blocks in the code. You can either just work in the ipynb OR you can work locally with the various files in this folder. If you don't have a GPU on your machine you will need to upload the ipynb to [Google Colaboratory](https://colab.google/) and work there.

## Simulation Description
Assume the world is split into `NUM_REGIONS` different regions. Each region is filled with `COMMUNITIES_PER_NUM_SPECIES` different communities of each of the `NUM_SPECIES` different species. Each of the communities is intialized with this information and a `population` and a `growth_rate` which is all packed into a `Species_Community` struct. Note that the struct also contains additional variables which you may or may not need to use depending upon your implementation (aka you explicitly don't need to use all of them, I just put them there to enable a bit of flexibility in how you implement things).

```
typedef struct {
    int population;        // the population of a speciies
    int food_collected;    // the food collected in the current time period
    int region_of_world;   // region of this species community
    int species_type;      // type of species for this species community
    float growth_rate;     // growth_rate for this species community
    bool flag;             // flag in case helpful to have one (you may not need this)
    int helperi;           // flag in case helpful to have one (you may not need this)
    float helperf;         // flag in case helpful to have one (you may not need this)
} Species_Community;
```

The simulation runs for `NUM_TIMEPERIODS`. At each time period all of the members of each species calls the `food_oracle` inorder for everyone to `gather_all_food`. After all food is collected we can `update_all_populations` based on the amount of food gathered. In order to do so we need to first `compute_local_population_share` which is the percentage of all species WITHIN A SINGLE REGION that belong to a given species. We can then use that to `update_community_population` for each community of each species based on 3 rules as listed in later sections of this document.

When the simulation is done it prints out the populations of the various communities of species.

## Functions You'll Need To Implement
All functions you need to implement are in `util.h` and that is the only file you need to submit to gradescope!

#### First we'll implement helper functions:

`update_community_population` (3 points)

For a given community, update its population based on the input `local_population_share` and the following three rules:
1. The change in population for a community is directly proportional to its growth_rate and local_population_share (aka the percent change is the multiplication of both the growth_rate and local_population_share). And make sure to round down!
2. If it has collected enough food to feed the population it grows, else it shrinks by the percent change determined above.
3. If the population drops below 5 it goes extinct (aka becomes 0).

`compute_local_population_share` (3 points)

Returns the population share for a given community. Population share is defined as the percentage of the total population in a region for a given species across all communities of all species.

#### Then we'll implement the overall population update step:

`update_all_populations` (5 points)

Updates the population for all communities of all species. Some quick hints/notes:
1. You will need to use compute_local_population_share and update_community_population
2. Make sure your logic is thread safe! Remember when you launch a kernel every line of the code is run by every thread in parallel!
3. Feel free to use helper functions if that makes your life easier!

#### Next we'll implement the food gathering step:

`gather_all_food` (3 points)

Each simualtion step we reset the food counts to 0 and then each member of the population tries to collect food using the food_oracle(). Try to maximize parallelism.

#### Then we'll implement the main kernel and function:

`population_dynamics_kernel` (3 points)

The kernel is the code that will run on the GPU. You want to make sure all NUM_TIME_PERIODS of gather_all_food and update_all_populations are run. To maximize performance on the GPU, you'll want to use shared memory to speed things up, but then make sure to save things back to RAM once you're done! Be careful that you copy the values from inside structs as needed!

Finally, we'll launch the main kernel from the main function:

`population_dynamics` (3 points)

Remember that we need to be careful about GPU vs. CPU memory! So set up GPU memory, run the kernel, and clean up GPU memory!

#### Example Output:
With the presets of 2,2,2,10,5 (the constatns in the helpers for the number of regions and species etc.) and any number of threads you should get:
```
ID[0]: of type [1]: in region [0]: had final population [11]
ID[1]: of type [1]: in region [0]: had final population [14]
ID[2]: of type [1]: in region [1]: had final population [237]
ID[3]: of type [0]: in region [1]: had final population [9]
ID[4]: of type [0]: in region [0]: had final population [97]
ID[5]: of type [0]: in region [0]: had final population [24]
ID[6]: of type [0]: in region [0]: had final population [5]
ID[7]: of type [1]: in region [1]: had final population [218]
```

## Submission
Once you are done, download and submit (or just submit if you are working locally) your `util.h` file to **Courseworks**! Sadly Gradescope does not support GPU autograders at this time.

## Notes and Hints
+ **DO NOT CHANGE FUNCTION DEFINITIONS** or you will break our grading scripts
+ See the syllabus for our course collaboration policy (long story short you are welcome to collaborate at a high level but please do not copy each others code).
+ Please reach out on Slack with any and all questions!

In [1]:
# make sure CUDA is installed
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
# make sure you have a GPU runtime (if this fails go to runtime -> change runtime type)
!nvidia-smi

Wed Feb 14 14:22:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
# CUDA in Jupyter helpers
!pip install nvcc4jupyter
%load_ext nvcc4jupyter
# to learn about how to do more fancy things with CUDA using this API see:
# https://nvcc4jupyter.readthedocs.io/en/latest/index.html

Source files will be saved in "/tmp/tmpzpy2ygyu".


In [4]:
%%cuda_group_save -n helpers.h -g default

#include <iostream>
#include <thread>
#include <vector>
#include <mutex>
#include <atomic>
#include <cstdlib>
#include <cmath>

// Some helpful constants
#define NUM_REGIONS 2
#define NUM_SPECIES 2
#define COMMUNITIES_PER_NUM_SPECIES 2
#define COMMUNITIES_PER_REGION (NUM_SPECIES*COMMUNITIES_PER_NUM_SPECIES)
#define TOTAL_COMMUNITIES (NUM_REGIONS*COMMUNITIES_PER_REGION)
#define MAX_STARTING_POPULATION 10
#define NUM_TIME_PERIODS 5
// for this pset we only need to consider 1 block :)
#define NUM_BLOCKS 1
#define NUM_THREADS (TOTAL_COMMUNITIES/NUM_BLOCKS)


// data structure to store information about each species
typedef struct {
    int population;        // the population of a speciies
    int food_collected;    // the food collected in the current time period
    int region_of_world;   // region of this species community
    int species_type;      // type of species for this species community
    float growth_rate;     // growth_rate for this species community
    bool flag;             // flag in case helpful to have one (you may not need this)
    int helperi;           // int storage in case helpful to have one (you may not need this)
    float helperf;         // float storage in case helpful to have one (you may not need this)
} Species_Community;

// food oracle function call
// call this with a community id to get a "random" amount of food back
// this represents one community member going out to get food
// we hardcode to 1 for determinism in testing but in theory should be random
__host__ __device__
int food_oracle(int community_id){return 1;};

// random range integer in range [0,max_range)
__host__
int rand_range(int max_range){return rand() % max_range;}
// random float in range [0,1]
__host__
float rand01(){return (float)rand() / (float)RAND_MAX;}

In [48]:
%%cuda_group_save -n util.h -g default

#include "helpers.h"

// function to simulate population change for one community of one species
//
// Note: 1) The change in population for a community is proportional to
//          its growth_rate and local_population_share (and round down)
//       2) If it has collected enough food to feed the population it grows, else it shrinks
//       3) If the population drops below 5 it goes extinct
//
// Hint: Casting to an int implicitly rounds down so if you round in another way make sure to round down!
__device__
void update_community_population(Species_Community *s_communities, int community_id, float local_population_share) {
  int change_in_pop = (int) s_communities[community_id].population*s_communities[community_id].growth_rate*local_population_share;
  bool growing = s_communities[community_id].food_collected >= s_communities[community_id].population;
  if(growing){
      s_communities[community_id].population += change_in_pop;
  }
  if(not growing){
      s_communities[community_id].population -= change_in_pop;
      if(s_communities[community_id].population<5){
          s_communities[community_id].population = 0;
      }
  }
}

// function to find the local population share for one community of one species
//
// Note: 1) Population share is defined as the percentage of population in a region
//          that is a given species across all communities of all species
//       2) You can assume this function is run by one thread if you'd like,
//          just then make sure you call it accordingly later!
//
// Hint: You may want to use some of the "helpful constants" and be careful about int vs float!
__device__
float compute_local_population_share(Species_Community *s_communities, int community_id){
  int species = s_communities[community_id].species_type;
  int population_of_species_in_region = 0;
  int population_other = 0;

  //loop through every community, check if it is correct region
  //count if it is of type species
  for (int looping_id = 0; looping_id < TOTAL_COMMUNITIES; looping_id++){
      if(s_communities[looping_id].region_of_world == s_communities[community_id].region_of_world){
          if(s_communities[looping_id].species_type == species){
              population_of_species_in_region += s_communities[looping_id].population;
          }
          else{
              population_other += s_communities[looping_id].population;
          }
      }
  }

  //then divide pop for species type / total pop of all communities in region
  float pop_share = static_cast<float>(population_of_species_in_region) / (population_of_species_in_region+population_other);
  return pop_share;
}

// Updates the population for all communities of all species (assumes food is already gathered)
//
// Note: 1) You will need to use compute_local_population_share and update_community_population
//       2) Use as many threads as possible but make sure your logic is thread safe!
//          This may have a race condition you need to watch out for (depending on your design)!
//       3) Feel free to use helper functions if that makes your life easier!
//       4) All other implementation details are up to you! (Don't worry if your design isn't perfect!)
__device__
void update_all_populations(Species_Community *s_communities){
  for(int i=threadIdx.x; i<TOTAL_COMMUNITIES; i+=blockDim.x){
    float pop_share = compute_local_population_share(s_communities, i);
    update_community_population(s_communities,i,pop_share);
  }
  __syncthreads();
}

// function to simulate food gathering
//
// Note: 1) Total food is always reset to 0 for each call and each member of the population tries to collect food
//       2) Please use food_oracle() to get a new amount of food for each member of the population
//       3) You can allocate threads to communites however you want!
//       3) All other implementation details are up to you! (Don't worry if your design isn't perfect!)
__device__
void gather_all_food(Species_Community *s_communities) {


  for(int i=threadIdx.x; i<TOTAL_COMMUNITIES; i+=blockDim.x){
      s_communities[i].food_collected = 0;
      for(int j=0;j<s_communities[i].population;j++){
          s_communities[i].food_collected += food_oracle(i);
      }
  }
    //for every community in s_communities
      // call food_oracle(community_id) once for each member of population
      // then add to s_communities[community_id].food_collected
      //definitely something parallelizable in the calling food oracle but alas, we can optimize later
}

// the main kernel that computes the population dynamics over time
//
// Hints: 1) using shared memory may speed things up
//           but then make sure to save things back to RAM
//        2) make sure you do all NUM_TIME_PERIODS of gather_all_food
//           and update_all_populations
__global__
void population_dynamics_kernel(Species_Community *d_communities){
  for(int i=0;i<NUM_TIME_PERIODS;i++){
      gather_all_food(d_communities);
      update_all_populations(d_communities);
  }
}

// the main function
//
// Hints: set up GPU memory, run the kernel, clean up GPU memory
__host__
void population_dynamics(Species_Community *h_communities){
  Species_Community *d_communities;
  cudaMalloc(&d_communities, TOTAL_COMMUNITIES*sizeof(Species_Community));
  cudaMemcpy(d_communities,h_communities,TOTAL_COMMUNITIES*sizeof(Species_Community),cudaMemcpyHostToDevice);

  population_dynamics_kernel<<<1,10>>>(d_communities); // launch the GPU code
  cudaDeviceSynchronize(); // wait until the GPU is done

  cudaMemcpy(h_communities,d_communities,TOTAL_COMMUNITIES*sizeof(Species_Community),cudaMemcpyDeviceToHost);
  cudaFree(d_communities);
}

In [49]:
%%cuda_group_save -n run.cu -g default

#include "util.h"

__host__
int main() {
  // initialize random and data
  srand(1337);
  Species_Community h_communities[TOTAL_COMMUNITIES];
  for (int community_id = 0; community_id < TOTAL_COMMUNITIES; community_id++){
    h_communities[community_id].population = rand_range(MAX_STARTING_POPULATION) + 5;
    h_communities[community_id].region_of_world = rand_range(NUM_REGIONS);
    h_communities[community_id].species_type = rand_range(NUM_SPECIES);
    h_communities[community_id].growth_rate = rand01();
  }

  for (int community_id = 0; community_id < TOTAL_COMMUNITIES; community_id++){
    std::cout << "ID[" << community_id << "]: of type [" << h_communities[community_id].species_type <<
                 "]: in region [" << h_communities[community_id].region_of_world << "]: had initial population [" <<
                 h_communities[community_id].population << "]" << std::endl;
  }

  // the main function
  population_dynamics(h_communities);

  // print the final populations
  std::cout << "\n---------\n---------\n\n";
  for (int community_id = 0; community_id < TOTAL_COMMUNITIES; community_id++){
    std::cout << "ID[" << community_id << "]: of type [" << h_communities[community_id].species_type <<
                 "]: in region [" << h_communities[community_id].region_of_world << "]: had final population [" <<
                 h_communities[community_id].population << "]" << std::endl;
  }
  return 0;
}

In [50]:
%cuda_group_run -g default

ID[0]: of type [1]: in region [0]: had initial population [6]
ID[1]: of type [1]: in region [0]: had initial population [9]
ID[2]: of type [1]: in region [1]: had initial population [13]
ID[3]: of type [0]: in region [1]: had initial population [9]
ID[4]: of type [0]: in region [0]: had initial population [10]
ID[5]: of type [0]: in region [0]: had initial population [5]
ID[6]: of type [0]: in region [0]: had initial population [5]
ID[7]: of type [1]: in region [1]: had initial population [12]

---------
---------

ID[0]: of type [1]: in region [0]: had final population [11]
ID[1]: of type [1]: in region [0]: had final population [14]
ID[2]: of type [1]: in region [1]: had final population [237]
ID[3]: of type [0]: in region [1]: had final population [9]
ID[4]: of type [0]: in region [0]: had final population [97]
ID[5]: of type [0]: in region [0]: had final population [24]
ID[6]: of type [0]: in region [0]: had final population [5]
ID[7]: of type [1]: in region [1]: had final populati

If you didn't change any parameters (besides the number of threads) then you should get:
```
ID[0]: of type [1]: in region [0]: had final population [11]
ID[1]: of type [1]: in region [0]: had final population [14]
ID[2]: of type [1]: in region [1]: had final population [237]
ID[3]: of type [0]: in region [1]: had final population [9]
ID[4]: of type [0]: in region [0]: had final population [97]
ID[5]: of type [0]: in region [0]: had final population [24]
ID[6]: of type [0]: in region [0]: had final population [5]
ID[7]: of type [1]: in region [1]: had final population [218]
```